### Preprocessing Data

In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import cv2
import random
import os
import pandas as pd
from PIL import Image
from skimage import exposure
from scipy.ndimage import gaussian_filter
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight

2024-04-11 04:23:47.612044: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 04:23:47.615384: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 04:23:47.656248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 04:23:48.611221: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [6]:
# Load the dataset
data = pd.read_csv("../../aws_s3/DDR+APTOS_TRAIN_TEST/train_test_DDR_APTOS.csv")

filtered_data = data[data["Data_source"] == "DDR"]
# Split the dataset into training and testing sets
train_data = filtered_data[filtered_data["Split"] == "Train"]
test_data = filtered_data[filtered_data["Split"] == "Test"]

In [7]:
# Define output directories
luminosity_output_dir_train = "luminosity_images_train/"
cropped_output_dir_train = "cropped_images_train/"
normalized_output_dir_train = "normalized_images_train/"
luminosity_output_dir_test = "luminosity_images_test/"
cropped_output_dir_test = "cropped_images_test/"
normalized_output_dir_test = "normalized_images_test/"

# Create directories if they don't exist
os.makedirs(luminosity_output_dir_train, exist_ok=True)
os.makedirs(cropped_output_dir_train, exist_ok=True)
os.makedirs(normalized_output_dir_train, exist_ok=True)
os.makedirs(luminosity_output_dir_test, exist_ok=True)
os.makedirs(cropped_output_dir_test, exist_ok=True)
os.makedirs(normalized_output_dir_test, exist_ok=True)

In [8]:
# luminosity and noise removal function
def luminosity_image(image_path, output_dir):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)

    # Merge back LAB channels
    lab_eq = cv2.merge((l_eq, a, b))
    enhanced_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    # Image noise removal using Gaussian filter
    filtered_image = cv2.GaussianBlur(enhanced_image, (5, 5), 0)

    # Save the preprocessed image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, filtered_image)

    return output_path

In [9]:
def crop_image(img, threshold=20, resize_flag=False, desired_size=(512, 512)):
    img_np = np.array(img)
    x_dim = img_np.shape[0]
    y_dim = img_np.shape[1]
    pixel_sums = img_np.sum(axis=2)
    x_arr = pixel_sums.sum(axis=1)
    y_arr = pixel_sums.sum(axis=0)
    x_start = np.where(x_arr > threshold * y_dim)[0][0]
    x_end = np.where(x_arr > threshold * y_dim)[0][-1]
    y_start = np.where(y_arr > threshold * x_dim)[0][0]
    y_end = np.where(y_arr > threshold * x_dim)[0][-1]
    new_img = img_np[x_start:x_end, y_start:y_end]
    new_img = Image.fromarray(new_img)
    if resize_flag:
        new_img = new_img.resize(desired_size)
    return new_img

In [10]:
def normalize_image(img):
    # Convert image to numpy array
    img_np = np.array(img)

    # Calculate mean and standard deviation (std) channel-wise
    mean_channels = np.mean(img_np, axis=(0, 1))
    std_channels = np.std(img_np, axis=(0, 1))

    # Normalize each channel separately
    normalized_image = np.zeros_like(img_np, dtype=np.float32)
    for channel in range(img_np.shape[2]):
        normalized_image[:, :, channel] = (
            img_np[:, :, channel] - mean_channels[channel]
        ) / std_channels[channel]

    # Scale values to be within [0, 255]
    normalized_image = (
        (normalized_image - np.min(normalized_image))
        / (np.max(normalized_image) - np.min(normalized_image))
        * 255
    )

    # Clip and return the normalized image
    normalized_image = np.clip(normalized_image, 0, 255)
    return normalized_image.astype(np.uint8)

In [12]:
X_train = []
y_train = []


# Function to get the correct image path with extension
def get_image_path(base_dir, img_name):
    for ext in ["jpg", "png"]:
        if "jpg" in img_name or "png" in img_name:
            temp_path = f"{base_dir}/{img_name}"
        else:
            temp_path = f"{base_dir}/{img_name}.{ext}"
        if os.path.exists(temp_path):
            return temp_path
    return None  # Return None if no file is found


# Load and preprocess each image
for index, row in train_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/train_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Change Luminosity and do noise removal for the image
    luminosity_img_path = luminosity_image(image_path, luminosity_output_dir_train)

    # Crop and resize the image
    luminosity_img = Image.open(luminosity_img_path)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(512, 512)
    )

    # Save cropped image
    # if "jpg" in img_name:
    #     cropped_img_name = f"{img_name}_{index+1}.jpg"
    # elif "png" in img_name:
    #     cropped_img_name = f"{img_name}_{index+1}.png"
    cropped_img_name = f"{img_name}_{index+1}.jpg"
    cropped_img_path = os.path.join(cropped_output_dir_train, cropped_img_name)
    cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    # if "jpg" in img_name:
    #     normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    # elif "png" in img_name:
    #     normalized_img_name = f"norm_{img_name}_{index+1}.png"
    normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    normalized_img_path = os.path.join(normalized_output_dir_train, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))
    X_train.append(normalized_img)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

Corrupt JPEG data: 34 extraneous bytes before marker 0xd9


KeyboardInterrupt: 

In [ ]:
X_test = []
y_test = []

# Load and preprocess each image
for index, row in test_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/test_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Preprocess the image
    luminosity_img_path = luminosity_image(image_path, luminosity_output_dir_test)

    # Crop and resize the image
    luminosity_img = Image.open(luminosity_img_path)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(512, 512)
    )

    # Save cropped image
    cropped_img_name = f"{img_name}_{index+1}.jpg"
    cropped_img_path = os.path.join(cropped_output_dir_test, cropped_img_name)
    cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    normalized_img_path = os.path.join(normalized_output_dir_test, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))

    # Append to X_test and y_test
    X_test.append(normalized_img)
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

### CNN512 Model

In [ ]:
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    BatchNormalization,
    ZeroPadding2D,
    Activation,
    Dropout,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# Define the CNN512 model architecture
model = Sequential()

# Input Layer (Zero Padding)
model.add(ZeroPadding2D(padding=(2, 2), input_shape=(512, 512, 3)))

# Layer 1, 2, 3
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 4
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 5, 6, 7
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 8
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 9, 10, 11
model.add(Conv2D(96, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 12
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 13, 14, 15
model.add(Conv2D(96, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 16
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 17, 18, 19
model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 20
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 21, 22, 23
model.add(Conv2D(200, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 24
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 25
model.add(Flatten())

# Layer 26 (Dropout)
model.add(Dropout(0.5))

# Layer 27, 28, 29
model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 30 (Dropout)
model.add(Dropout(0.5))

# Layer 31, 32, 33
model.add(Dense(500))
model.add(BatchNormalization())
model.add(Activation("relu"))

# Layer 34 (Dropout)
model.add(Dropout(0.5))

# Layer 35
model.add(Dense(4, activation="softmax"))  # Assuming 4 classes for Retinopathy grade

In [ ]:
from tensorflow.keras.optimizers import SGD

# base learning rate
base_learning_rate = 1e-4
# maximum learning rate
max_learning_rate = 1e-2

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [ ]:
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=4,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

In [ ]:
print("Test dataset evaluation")
model.evaluate(X_test, y_test)